https://www.youtube.com/watch?v=ujId4ipkBio&t=687s

https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524

In [67]:
from google.colab import files
uploaded =files.upload()

In [68]:
import pandas as pd
import numpy as np

import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


In [137]:
tdf= pd.read_csv('combined_data.csv')

In [138]:
tdf.head()

,index,Content,Time
0,0,"@haltman Little do they know, birds aren't rea...",2022-03-11 19:41:00
1,1,@waitbutwhy What if one atom at a time in your...,2022-03-11 00:31:00
2,2,@WholeMarsBlog @Erdayastronaut @CopSub This we...,2022-03-10 05:44:00
3,3,@FedorovMykhailo @OMarkarova You're welcome. W...,2022-03-09 21:49:00
4,4,"@PPathole @SpaceX Optimized, fully-reusable St...",2022-03-09 21:41:00


In [151]:
#i will create a sample data to have faster runtime etc and make it easier for me.


df_s=tdf.copy()
#df_s = tdf.loc[0:120,:]
df_s=tdf.drop('index',axis=1)
df_s['Content'].value_counts()

@BillyM2k                                                                                                                                     18
@nichegamer                                                                                                                                   11
@flcnhvy True                                                                                                                                 10
@TheBabylonBee                                                                                                                                 9
@engineers_feed                                                                                                                                9
                                                                                                                                              ..
@PPathole That's the goal!                                                                                                        

In [148]:
#Cleaning the tweets

def cleanTxt(text):
  text = re.sub('@[A-Za-z0-9]+' , '', text) #remove the @mentions
  text = re.sub('https?:\/\/\S+' , '', text) #remove the hyperlinks
  text = re.sub(r'#', '', text) #removing #tags
  text = re.sub(r'RT[\s]+', '', text) #removing RT(ReTweets)

  return text

In [149]:
df_s['Content'] = df_s['Content'].apply(cleanTxt)

TypeError: ignored

The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

Polarity is a float which lies between the ranges of [-1.0,1.0].
-1 defines a negative sentiment
1 defines a positive sentiment.
Subjectivity lies between [0.0,1.0].
0.0 is very objective and 1.0 is very subjective.
If subjectivity < 0.5, the sentence is more subjective than objective and vice versa.

In [135]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

#Create 2 new columns
df_s['Subjectivity'] = df_s['Content'].apply(getSubjectivity)
df_s['Polarity'] = df_s['Content'].apply(getPolarity)

df_s.head()

TypeError: ignored

In [ ]:
#Plot Word Cloud
allWords = ''.join([twts for twts in df_s['Content']])
wordCloud = WordCloud(width=500, height=300,random_state=21,max_font_size=119).generate(allWords)

plt.imshow(wordCloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
#create a function to compute the negative, neutral and positive analysis

def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score==0:
    return 'Neutral'
  elif score > 0:
    return 'Positive'

def getSignal(analysis):
  if analysis == 'Negative':
    return -1
  elif analysis == 'Neutral':
    return 0
  elif analysis == 'Positive':
    return 1

df_s['Analysis'] = df_s['Polarity'].apply(getAnalysis)
df_s['Signal']= df_s['Analysis'].apply(getSignal)

df_s


In [ ]:
#Print all positive tweets
j=1
sortedDF = df_s.sort_values(by=['Polarity']) #sorts the df by polarity

for i in range(0, sortedDF.shape[0]):   #number of row in our df
  if (sortedDF['Analysis'][i]== 'Positive'):          #if sorted  analysis at position 1 is positive
    print(str(j) + sortedDF['Content'][i] +' '+ '1')
    print()
    j=j+1
  elif (sortedDF['Analysis'][i]== 'Negative'):
    print(str(j) + sortedDF['Content'][i] +' '+ '-1')
    print()
    j=j+1
  elif (sortedDF['Analysis'][i]== 'Neutral'):
    print(str(j) + sortedDF['Content'][i] +' '+ '0')
    print()
    j=j+1

In [ ]:
#Set time as index. We can later use the acsending as per need

from datetime import datetime

# Write a format string to parse s
fmt = '%Y-%m-%d %H:%M:%S'

# Create a datetime object d


df_s['Time']=pd.to_datetime(df_s['Time'], format=fmt)

df_s = df_s.set_index(df_s['Time'])
df_s = df_s.sort_index(ascending=True)
df_s = df_s.drop('index', axis=1)
df_s

In [ ]:
# if the tweet is between 9-17 sum up the subjectivtity/polarity and establish the signal
#if the tweet is between 17-9 sum up and establish a signal 

In [ ]:

time_column = df_s['Time']
one_day = []

print(d)
day

In [ ]:
print(df_s['2022-03-06']['Analysis'])


In [ ]:
#how to get mean of the signal on each day in df

signal_day=df_s['Signal'].resample('D').mean()
print(signal_day['2022-03-06'])


#starting from 8:59:59 morning to 17:00:01 same day
#starting from 17:00:03 to 8:59:59 next day 


signal_day=df_s['Signal'].resample('H').mean()
print(signal_day['2022-03-06 08:59:59' : '2022-03-06 17:00:01'])
print(signal_day['2022-03-06 17:00:02' : '2022-03-07 08:59:58'])
print(signal_day['2022-03-07 08:59:59' : '2022-03-07 17:00:01'])
print(signal_day['2022-03-07 17:00:02' : '2022-03-08 08:59:58'])

 #play with the date and it will give you the signals you need

In [ ]:
%matplotlib inline
signal_day.plot()